<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Masterproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle

In [2]:
!git clone https://github.com/aim56009/AI-MA_project.git

Cloning into 'AI-MA_project'...
remote: Enumerating objects: 1809, done.
remote: Counting objects: 100% (1809/1809), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 1809 (delta 1756), reused 1780 (delta 1747), pack-reused 0
Receiving objects: 100% (1809/1809), 3.86 MiB | 12.46 MiB/s, done.
Resolving deltas: 100% (1756/1756), done.
Checking out files: 100% (1846/1846), done.


In [13]:
PATH_TO_DATA = "AI-MA_project/pianoroll"

In [14]:
with open(PATH_TO_DATA + "/voice_0/voice_0_001.pkl",'rb') as f:
    loaded_obj = pickle.load(f)
    print(loaded_obj.shape)

(128, 756)


In [15]:
loaded_obj

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Model

In [33]:
class PKSpell_single(nn.Module):

    def __init__(self, pitch_to_ix, output_dim, input_dim=17, hidden_dim=300, rnn_depth=1, cell_type="GRU"):
        super(PKSpell_single, self).__init__()

        self.n_out = output_dim
        
        
        rnn_cell = nn.GRU

        self.rnn = rnn_cell(
            input_size=input_dim,
            hidden_size= hidden_dim,
            num_layers=rnn_depth)


        # Output layers. The input will be two times
        self.top_layer_voice_0 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_1 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_2 = nn.Linear(hidden_dim, self.n_out)
        self.top_layer_voice_3 = nn.Linear(hidden_dim, self.n_out)

        # Loss function that we will use during training.
        self.loss = nn.CrossEntropyLoss(reduction="mean")


    def compute_outputs(self, sentences, sentences_len):
        
        rnn_out = self.rnn(sentences)
        
        out_0 = self.top_layer_voice_0(rnn_out)
        out_1 = self.top_layer_voice_1(rnn_out)
        out_2 = self.top_layer_voice_2(rnn_out)
        out_3 = self.top_layer_voice_3(rnn_out)
        

        return out_0, out_1, out_2, out_3

    def forward(self, sentences, v0, v1, v2, v3, sentences_len):
        # First computes the predictions, and then the loss function.

        # Compute the outputs. The shape is (max_len, n_sentences, n_labels).
        scores_0, scores_1, scores_2, scores_3 = self.compute_outputs(sentences, sentences_len)

        # Flatten the outputs and the labels, to compute the loss.
        # The input to this loss needs to be one 2-dimensional and one 1-dimensional tensor.
        score_0  = scores_0.view(-1, self.n_out)
        score_1  = scores_1.view(-1, self.n_out)
        score_2  = scores_2.view(-1, self.n_out)
        score_3  = scores_3.view(-1, self.n_out)


        v0 = v0.view(-1)
        v1 = v1.view(-1)
        v2 = v2.view(-1)
        v3 = v3.view(-1)

        
        loss = self.loss_pitch(score_0, v0) + self.loss_pitch(score_1, v1) + self.loss_pitch(score_2, v2) + self.loss_pitch(score_3, v3)
        
        return loss

    def predict(self, sentences, sentences_len):
        # Compute the outputs from the linear units.
        scores_0, scores_1, scores_2, scores_3 = self.compute_outputs(sentences, sentences_len)

        # Select the top-scoring labels. The shape is now (max_len, n_sentences).
        predicted_0 = scores_0.argmax(dim=2)
        predicted_1 = scores_1.argmax(dim=2)
        predicted_2 = scores_2.argmax(dim=2)
        predicted_3 = scores_3.argmax(dim=2)

        return (
            [
                predicted_0[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_1[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_2[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
            [
                predicted_3[: int(l), i].cpu().numpy()
                for i, l in enumerate(sentences_len)
            ],
        )

